In [0]:
!pip install patool

     |████████████████████████████████| 81kB 1.2MB/s 


In [0]:
!pip install pyunpack

  Created wheel for pyunpack: filename=pyunpack-0.1.2-cp36-none-any.whl size=5163 sha256=e0e7e5c2420b13dd8bc4505733ed8cc8ed1f42987743051dd32ff8d0336d152e
  Stored in directory: /root/.cache/pip/wheels/af/44/08/60613970881e542c0baad1f2dea5ed8e6716bc573f49197b7e
Successfully built pyunpack


In [0]:
from pyunpack import Archive
Archive('/content/drive/My Drive/Information_Retrieval/FIRE_Dataset_EN_2010.rar').extractall('.')
Archive('/content/FIRE_Dataset_EN_2010/English-Data.tgz').extractall('.')
Archive('/content/FIRE_Dataset_EN_2010/en.qrels.76-125.2010.txt.gz').extractall('.')

In [0]:
f = open("/content/en.qrels.76-125.2010.txt", "r")
l = f.readlines()
query_rel_doc = dict()
for i in l:
    q_list = i.split()
    if i[len(i) - 2] == '1':
        query_num = q_list[0]
        if query_rel_doc.get(query_num) == None:
            query_rel_doc[query_num] = [ q_list[2]]
        else:
            li = query_rel_doc[query_num]
            li.append(q_list[2])

In [0]:
import pickle

with open('/content/drive/My Drive/Information_Retrieval/query_rel_doc.pickle', 'wb') as handle:
    pickle.dump(query_rel_doc, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:
import pickle
handle =  open('/content/drive/My Drive/Information_Retrieval/query_rel_doc.pickle', 'rb') 
query_rel_doc = pickle.load(handle)
handle.close()

In [0]:
import pandas as pd
data = pd.read_csv('/content/drive/My Drive/Information_Retrieval/assignment_word2vec_dataframe.csv')

In [0]:
import pandas as pd
preprocessed_data = pd.read_csv('/content/drive/My Drive/Information_Retrieval/assignment_preproc_word2vec_dataframe.csv')

In [5]:
preprocessed_data.head()

,DOCNO,TEXT
0,1050423_bengal_index.utf8,the telegraph calcutta bengal saturda...
1,1050303_bengal_story_4447924.utf8,the telegraph calcutta bengal thursda...
2,1050906_bengal_story_5203263.utf8,the telegraph calcutta bengal tuesday...
3,1051219_bengal_story_5618750.utf8,the telegraph calcutta bengal monday ...
4,1051231_bengal_story_5665519.utf8,the telegraph calcutta bengal saturda...


In [0]:
preprocessed_data.columns

Index(['DOCNO', 'TEXT'], dtype='object')

In [0]:
import pandas as pd
query_data = pd.read_csv("/content/drive/My Drive/Information_Retrieval/query_data.csv")

In [18]:
query_data.head()

,query_no,query_desc
0,76,reasons behind the protests by meena leaders ...
1,77,attacks by hezbollah guerrillas on indian and...
2,78,conflict between ashok singhal the president...
3,79,plans to build a road from china to mount eve...
4,80,initiation of legal proceedings against advan...


# **TF-IDF + ROCCHIO**

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=0)
document_vectors = vectorizer.fit_transform(preprocessed_data['TEXT'].values)

In [0]:
document_vectors.shape

(125586, 235155)

In [0]:
import numpy as np
sum_of_document_vectors = np.zeros((1, 235155))

for i in range(document_vectors.shape[0]):
  sum_of_document_vectors+=document_vectors[i].toarray()


In [0]:
np.save('/content/drive/My Drive/Information_Retrieval/sum_of_document_vectors.npy',sum_of_document_vectors,allow_pickle=True)

In [0]:
query_data.columns

Index(['query_no', 'query_desc'], dtype='object')

In [0]:
query_vectors = vectorizer.transform(query_data['query_desc'].values)

In [0]:
query_vectors_rel_feedback = []
#Implementing rocchio algorithm 
for i in range(50):
  query_num = query_data['query_no'][i]
  document_ids = query_rel_doc[str(query_num)]
  sum_of_relevant_doc_vectors = np.zeros((1, 235155))
  for doc_id in document_ids:
    for j in range(preprocessed_data.shape[0]):
      if doc_id == preprocessed_data['DOCNO'][j]:
        sum_of_relevant_doc_vectors+=document_vectors[i].toarray()
  avg_rel_doc_vectors = sum_of_relevant_doc_vectors/len(document_ids)
  avg_irrelevant_vectors = (sum_of_document_vectors - sum_of_relevant_doc_vectors)/(preprocessed_data.shape[0] - len(document_ids))
  # alpha = 1, beta = 0.75 and gamma = 0.15 
  final_query_vector = query_vectors[i].toarray()+ 0.75 * avg_rel_doc_vectors - 0.15 * avg_irrelevant_vectors
  final_query_vector = final_query_vector.clip(min = 0)
  query_vectors_rel_feedback.append(final_query_vector)
 

In [0]:
import pickle

with open('/content/drive/My Drive/Information_Retrieval/query_vectors_rel_feedback.pickle', 'wb') as handle:
    pickle.dump(query_vectors_rel_feedback, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
#performing similarity check between queries and documents and finding top 5 relevant documents
similarity_values = []
query_relevant_documents = dict()
for k in range(50):
    query_vector = query_vectors_rel_feedback[k]
    similarity_values = []
    for i in range(preprocessed_data.shape[0]):
        doc_vector = document_vectors[i].toarray()
        try:
          value = cosine_similarity(query_vector,doc_vector)[0][0]
        except:
          break
        similarity_values.append([value,i])
   
    similarity_values.sort(reverse = True)
    top_5 = similarity_values[0:5]
    query_num = query_data['query_no'][k]
    doc_indices = [ i[1] for i in top_5 ]
    relevant_docs = data['TEXT'][doc_indices]
    relevant_doc_ids = data['DOCNO'][doc_indices]
    query_relevant_documents[query_num] = [relevant_doc_ids,relevant_docs]

In [0]:
query_precision_recall_f1_score = []
for i in query_relevant_documents.keys():
    l = []
    print("Query Number: ",i)
    l.append(i)
    print("Documents retrieved using QUERY ROCCHIO(TF-IDF): ")
    print(query_relevant_documents[i][0])
    print("\nRelevant documents: ")
    for q in query_rel_doc[str(i)]:
        print(q)
    retrieved = set(query_relevant_documents[i][0])
    act_docs = set(query_rel_doc[str(i)])
    precision = len(retrieved.intersection(act_docs))/5
    recall = len(retrieved.intersection(act_docs))/len(act_docs)
    f1_score = 0
    if precision+recall != 0:
      f1_score = 2*precision*recall/(precision+recall)
    l.extend([precision,recall,f1_score])
    query_precision_recall_f1_score.append(l)
    print("\nNumber of relevant documents retrieved: ",len(retrieved.intersection(act_docs)))
    print("Relevant documents retrieved using QUERY ROCCHIO(TF-IDF):")
    for d in retrieved.intersection(act_docs):
        print(d)
    print("\n")
    print("*"*100)

Query Number:  76
Documents retrieved using QUERY EXPANSION(TF-IDF): 
0                 1050423_bengal_index.utf8
1875      1050423_bengal_story_4651659.utf8
2338      1050426_bengal_story_4662636.utf8
59415    1060525_opinion_story_6230941.utf8
13204     1050422_nation_story_4643866.utf8
Name: DOCNO, dtype: object

Relevant documents: 
1070530_nation_story_7849973.utf8
1070602_nation_story_7865940.utf8
1070603_nation_story_7869357.utf8
1070611_nation_story_7906812.utf8

Number of relevant documents retrieved:  0
Relevant documents retrieved using QUERY EXPANSION(TF-IDF):


****************************************************************************************************
Query Number:  77
Documents retrieved using QUERY EXPANSION(TF-IDF): 
1           1050303_bengal_story_4447924.utf8
2210        1050316_bengal_story_4498474.utf8
597         1050226_bengal_story_4427183.utf8
31819    1050325_frontpage_story_4535852.utf8
573         1050321_bengal_story_4518451.utf8
Name: DOCNO, dtype

In [0]:
avg_precision = 0
avg_recall = 0
avg_f1_score = 0

for query_no,precision,recall,f1_score in query_precision_recall_f1_score:
  avg_precision+=precision
  avg_recall+=recall
  avg_f1_score+=f1_score

print('Average Precison: {} Average Recall: {} Average F1-Score: {}'.format(avg_precision/len(query_precision_recall_f1_score),
                                                                            avg_recall/len(query_precision_recall_f1_score),
                                                                            avg_f1_score/len(query_precision_recall_f1_score)))

Average Precison: 0.284 Average Recall: 0.15509316130618703 Average F1-Score: 0.18249150032374165


# **WORD2VEC + ROCCHIO**

In [0]:
from gensim.test.utils import datapath
from gensim import utils

class MyCorpus(object):
    """An interator that yields sentences (lists of str)."""
    def __iter__(self): 
        preprocessed_text_column = preprocessed_data['TEXT'].values
        for line in preprocessed_text_column:
            # assume there's one document per line, tokens separated by whitespace
            yield utils.simple_preprocess(line)

In [0]:
import gensim.models
#skipgram model
sentences = MyCorpus()
skipGramModel = gensim.models.Word2Vec(sentences=sentences,sg = 1,workers=4)

In [0]:
query_vectors = []
for k in range(50):

    #generating vector for each query by summing word vectors
    query = query_data['query_desc'][k]
    query_words = query.split()
    query_vector = np.zeros(100)
    count = 0
    for l in range(len(query_words)):
      try:
        query_vector+=skipGramModel.wv.word_vec(query_words[l])
        count+=1
      except:
        pass
    if count > 1:
      query_vector/=count
    query_vectors.append(query_vector)


In [0]:
doc_vectors = []
for k in range(preprocessed_data.shape[0]):
#generating vector for each document by summing word vectors
    doc = preprocessed_data['TEXT'][k]
    doc_words = doc.split()
    doc_vector = np.zeros(100)
    count = 0
    for l in range(len(doc_words)):
      try:
        doc_vector+=skipGramModel.wv.word_vec(doc_words[l])
        count+=1
      except:
        pass
    if count > 1:
      doc_vector/=count
    doc_vectors.append(doc_vector)

In [0]:
import numpy as np
sum_of_document_vectors = np.zeros(100)
for i in range(len(doc_vectors)):
  sum_of_document_vectors+=doc_vectors[i]

In [0]:
query_vectors_rel_feedback = []
query_vectors_rel_zero_gamma_feedback = []
#Implementing rocchio algorithm 
for i in range(50):
  query_num = query_data['query_no'][i]
  document_ids = query_rel_doc[str(query_num)]
  sum_of_relevant_doc_vectors = np.zeros(100)
  for doc_id in document_ids:
    for j in range(preprocessed_data.shape[0]):
      if doc_id == preprocessed_data['DOCNO'][j]:
        sum_of_relevant_doc_vectors+=doc_vectors[i]
  avg_rel_doc_vectors = sum_of_relevant_doc_vectors/len(document_ids)
  avg_irrelevant_vectors = (sum_of_document_vectors - sum_of_relevant_doc_vectors)/(preprocessed_data.shape[0] - len(document_ids))
  
  # alpha = 1, beta = 0.75 and gamma = 0.15 
  final_query_vector = query_vectors[i]+ 0.75 * avg_rel_doc_vectors - 0.15 * avg_irrelevant_vectors
  query_vectors_rel_feedback.append(final_query_vector)

  final_query_vector_zero_gamma = query_vectors[i]+ 0.75 * avg_rel_doc_vectors
  query_vectors_rel_zero_gamma_feedback.append(final_query_vector_zero_gamma)


In [0]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
#performing similarity check between queries and documents and finding top 5 relevant documents
similarity_values = []
similarity_values_zero_gamma = []
query_relevant_documents = dict()
query_relevant_documents_zero_gamma = dict()

for k in range(50):
    query_vector = query_vectors_rel_feedback[k]
    query_vector_zero_gamma = query_vectors_rel_zero_gamma_feedback[k]
    similarity_values = []
    for i in range(preprocessed_data.shape[0]):  
        try:
          value = cosine_similarity([query_vector],[doc_vectors[i]])[0][0]
          value_zero_gamma = cosine_similarity([query_vector_zero_gamma],[doc_vectors[i]])[0][0]
        except:
          break
        similarity_values.append([value,i])
        similarity_values_zero_gamma.append([value_zero_gamma,i])
   
    query_num = query_data['query_no'][k]

    similarity_values.sort(reverse = True)
    similarity_values_zero_gamma.sort(reverse = True)

    top_5 = similarity_values[0:5]
    top_5_zero_gamma = similarity_values_zero_gamma[0:5]

    doc_indices = [ i[1] for i in top_5 ]
    doc_indices_zero_gamma = [ i[1] for i in top_5_zero_gamma ]

    relevant_docs = data['TEXT'][doc_indices]
    relevant_docs_zero_gamma = data['TEXT'][doc_indices_zero_gamma]

    relevant_doc_ids = data['DOCNO'][doc_indices]
    relevant_doc_ids_zero_gamma =  data['DOCNO'][doc_indices_zero_gamma]

    query_relevant_documents[query_num] = [relevant_doc_ids,relevant_docs]
    query_relevant_documents_zero_gamma[query_num] = [relevant_doc_ids_zero_gamma,relevant_docs_zero_gamma]

In [0]:
query_precision_recall_f1_score = []
for i in query_relevant_documents.keys():
    l = []
    print("Query Number: ",i)
    l.append(i)
    print("Documents retrieved using QUERY ROCCHIO(word2vec): ")
    print(query_relevant_documents[i][0])
    print("\nRelevant documents: ")
    for q in query_rel_doc[str(i)]:
        print(q)
    retrieved = set(query_relevant_documents[i][0])
    act_docs = set(query_rel_doc[str(i)])
    precision = len(retrieved.intersection(act_docs))/5
    recall = len(retrieved.intersection(act_docs))/len(act_docs)
    f1_score = 0
    if precision+recall != 0:
      f1_score = 2*precision*recall/(precision+recall)
    l.extend([precision,recall,f1_score])
    query_precision_recall_f1_score.append(l)
    print("\nNumber of relevant documents retrieved: ",len(retrieved.intersection(act_docs)))
    print("Relevant documents retrieved using QUERY ROCCHIO(word2vec):")
    for d in retrieved.intersection(act_docs):
        print(d)
    print("\n")
    print("*"*100)

Query Number:  76
Documents retrieved using QUERY ROCCHIO(word2vec): 
508                  1050513_bengal_index.utf8
100978    1070211_frontpage_story_7377565.utf8
54205        1061203_nation_story_7084990.utf8
30347     1050626_frontpage_story_4916195.utf8
18414       1050811_foreign_story_5101180.utf8
Name: DOCNO, dtype: object

Relevant documents: 
1070530_nation_story_7849973.utf8
1070602_nation_story_7865940.utf8
1070603_nation_story_7869357.utf8
1070611_nation_story_7906812.utf8

Number of relevant documents retrieved:  0
Relevant documents retrieved using QUERY ROCCHIO(word2vec):


****************************************************************************************************
Query Number:  77
Documents retrieved using QUERY ROCCHIO(word2vec): 
55225    1060717_foreign_story_6488684.utf8
17074    1051229_foreign_story_5657775.utf8
57066    1060820_foreign_story_6632582.utf8
57716    1060720_foreign_story_6501311.utf8
55425    1060814_foreign_story_6609200.utf8
Name: DOCNO, 

In [0]:
query_precision_recall_f1_score_zero_gamma = []
for i in query_relevant_documents_zero_gamma.keys():
    l = []
    print("Query Number: ",i)
    l.append(i)
    print("Documents retrieved using QUERY ROCCHIO(word2vec) gamma = 0: ")
    print(query_relevant_documents[i][0])
    print("\nRelevant documents: ")
    for q in query_rel_doc[str(i)]:
        print(q)
    retrieved = set(query_relevant_documents[i][0])
    act_docs = set(query_rel_doc[str(i)])
    precision = len(retrieved.intersection(act_docs))/5
    recall = len(retrieved.intersection(act_docs))/len(act_docs)
    f1_score = 0
    if precision+recall != 0:
      f1_score = 2*precision*recall/(precision+recall)
    l.extend([precision,recall,f1_score])
    query_precision_recall_f1_score_zero_gamma.append(l)
    print("\nNumber of relevant documents retrieved: ",len(retrieved.intersection(act_docs)))
    print("Relevant documents retrieved using QUERY ROCCHIO(word2vec) gamma = 0:")
    for d in retrieved.intersection(act_docs):
        print(d)
    print("\n")
    print("*"*100)

Query Number:  76
Documents retrieved using QUERY ROCCHIO(word2vec) gamma = 0: 
508                  1050513_bengal_index.utf8
100978    1070211_frontpage_story_7377565.utf8
54205        1061203_nation_story_7084990.utf8
30347     1050626_frontpage_story_4916195.utf8
18414       1050811_foreign_story_5101180.utf8
Name: DOCNO, dtype: object

Relevant documents: 
1070530_nation_story_7849973.utf8
1070602_nation_story_7865940.utf8
1070603_nation_story_7869357.utf8
1070611_nation_story_7906812.utf8

Number of relevant documents retrieved:  0
Relevant documents retrieved using QUERY ROCCHIO(word2vec) gamma = 0:


****************************************************************************************************
Query Number:  77
Documents retrieved using QUERY ROCCHIO(word2vec) gamma = 0: 
55225    1060717_foreign_story_6488684.utf8
17074    1051229_foreign_story_5657775.utf8
57066    1060820_foreign_story_6632582.utf8
57716    1060720_foreign_story_6501311.utf8
55425    1060814_foreign_st

In [0]:
avg_precision = 0
avg_recall = 0
avg_f1_score = 0

for query_no,precision,recall,f1_score in query_precision_recall_f1_score:
  avg_precision+=precision
  avg_recall+=recall
  avg_f1_score+=f1_score

print('Average Precison: {} Average Recall: {} Average F1-Score: {}'.format(avg_precision/len(query_precision_recall_f1_score),
                                                                            avg_recall/len(query_precision_recall_f1_score),
                                                                            avg_f1_score/len(query_precision_recall_f1_score)))

Average Precison: 0.26 Average Recall: 0.12299693342589052 Average F1-Score: 0.15242089208780835


In [0]:
avg_precision = 0
avg_recall = 0
avg_f1_score = 0

for query_no,precision,recall,f1_score in query_precision_recall_f1_score_zero_gamma:
  avg_precision+=precision
  avg_recall+=recall
  avg_f1_score+=f1_score

print('Average Precison: {} Average Recall: {} Average F1-Score: {}'.format(avg_precision/len(query_precision_recall_f1_score),
                                                                            avg_recall/len(query_precision_recall_f1_score),
                                                                            avg_f1_score/len(query_precision_recall_f1_score)))

Average Precison: 0.26 Average Recall: 0.12299693342589052 Average F1-Score: 0.15242089208780835


# **QUERY EXPANSION**

In [0]:
import nltk 
from nltk.corpus import wordnet 

#Performing query expansion by adding synonyms of words in each of the queries

nltk.download('wordnet')
list_of_expanded_queries = []
for i in range(query_data.shape[0]):
  query = query_data['query_desc'][i]
  query_no = query_data['query_no'][i]
  words_in_query = query.split()
  new_words_in_query = []
  new_words_in_query.extend(words_in_query)
  for word in words_in_query:
    count = 0
    for syn in wordnet.synsets(word): 
      if count <= 3:
        for l in syn.lemmas(): 
            new_words_in_query.append(l.name())
            count+=1

  expanded_query = " ".join(new_words_in_query)
  list_of_expanded_queries.append([query_no,expanded_query])

In [0]:
list_of_expanded_queries[0][1],query_data['query_desc'][0]

('reasons behind the protests by meena leaders against the inclusion of gurjars in the scheduled tribes reason ground reason reason understanding intellect buttocks nates arse butt backside bum buns can fundament hindquarters hind_end keister posterior prat rear rear_end rump stern seat tail tail_end tooshie tush bottom behind derriere fanny ass protest protestation protest objection dissent by past aside by away leadership leaders leader drawing_card loss_leader leader inclusion inclusion comprehension inclusion_body cellular_inclusion inclusion inch in indium In atomic_number_49 schedule schedule scheduled tribe folk tribe federation_of_tribes',
 ' reasons behind the protests by meena leaders against the inclusion of gurjars in the scheduled tribes  ')

In [0]:
expanded_query_dataframe = pd.DataFrame(data = list_of_expanded_queries,columns = ['query_no','query_desc'])

In [0]:
expanded_query_dataframe.to_csv('/content/drive/My Drive/Information_Retrieval/expanded_query_data.csv',index = False)

In [0]:
import pandas as pd
expanded_query_dataframe = pd.read_csv('/content/drive/My Drive/Information_Retrieval/expanded_query_data.csv')

In [9]:
expanded_query_dataframe.head()

,query_no,query_desc
0,76,reasons behind the protests by meena leaders a...
1,77,attacks by hezbollah guerrillas on indian and ...
2,78,conflict between ashok singhal the president o...
3,79,plans to build a road from china to mount ever...
4,80,initiation of legal proceedings against advani...


# **TF-IDF + QUERY EXPANSION + ROCCHIO**

---



In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=0)
document_vectors = vectorizer.fit_transform(preprocessed_data['TEXT'].values)

In [0]:
expanded_query_tfidf_vectors = vectorizer.transform(expanded_query_dataframe['query_desc'].values)

In [0]:
import numpy as np
expanded_query_tfidf_vectors = expanded_query_tfidf_vectors.toarray()
np.save('/content/drive/My Drive/Information_Retrieval/expanded_query_tfidf_vectors.npy',expanded_query_tfidf_vectors,allow_pickle=True)

In [0]:
import numpy as np
expanded_query_tfidf_vectors = np.load('/content/drive/My Drive/Information_Retrieval/expanded_query_tfidf_vectors.npy',allow_pickle=True)

In [0]:
expanded_query_tfidf_vectors.shape

(50, 235155)

In [0]:
import numpy as np
sum_of_document_vectors = np.zeros((1, 235155))

for i in range(document_vectors.shape[0]):
  sum_of_document_vectors+=document_vectors[i].toarray()


In [0]:
query_vectors_rel_feedback = []
query_vectors_rel_feedback_gamma_zero = []
#Implimenting rocchio algorithm 
for i in range(50):
  query_num = query_data['query_no'][i]
  document_ids = query_rel_doc[str(query_num)]
  sum_of_relevant_doc_vectors = np.zeros((1, 235155))
  
  for doc_id in document_ids:
    for j in range(preprocessed_data.shape[0]):
      if doc_id == preprocessed_data['DOCNO'][j]:
        sum_of_relevant_doc_vectors+=document_vectors[i].toarray()

  avg_rel_doc_vectors = sum_of_relevant_doc_vectors/len(document_ids)
  avg_irrelevant_vectors = (sum_of_document_vectors - sum_of_relevant_doc_vectors)/(preprocessed_data.shape[0] - len(document_ids))
  # alpha = 1, beta = 0.75 and gamma = 0.15 
  final_query_vector = expanded_query_tfidf_vectors[i]+ 0.75 * avg_rel_doc_vectors - 0.15 * avg_irrelevant_vectors
  final_query_vector = final_query_vector.clip(min = 0)

  final_query_vector_gamma_zero = expanded_query_tfidf_vectors[i]+ 0.75 * avg_rel_doc_vectors
  final_query_vector_gamma_zero = final_query_vector_gamma_zero.clip(min = 0)

  query_vectors_rel_feedback.append(final_query_vector)
  query_vectors_rel_feedback_gamma_zero.append(final_query_vector_gamma_zero)

In [0]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
#performing similarity check between queries and documents and finding top 5 relevant documents

query_relevant_documents = dict()
query_relevant_documents_zero_gamma = dict()
for k in range(50):
    query_vector = query_vectors_rel_feedback[k]
    query_vector_zero_gamma = query_vectors_rel_feedback_gamma_zero[k]
    similarity_values = []
    similarity_values_zero_gamma = []
    
    for i in range(preprocessed_data.shape[0]):
        doc_vector = document_vectors[i].toarray()
        try:
          value = cosine_similarity(query_vector,doc_vector)[0][0]
          value_zero_gamma = cosine_similarity(query_vector_zero_gamma,doc_vector)[0][0]
        except:
          break
        similarity_values.append([value,i])
        similarity_values_zero_gamma.append([value_zero_gamma,i])
   
    similarity_values.sort(reverse = True)
    similarity_values_zero_gamma.sort(reverse = True)

    top_5 = similarity_values[0:5]
    top_5_zero_gamma = similarity_values_zero_gamma[0:5]

    query_num = query_data['query_no'][k]

    doc_indices = [ i[1] for i in top_5 ]
    doc_indices_zero_gamma = [ i[1] for i in top_5_zero_gamma ]

    relevant_docs = data['TEXT'][doc_indices]
    relevant_docs_zero_gamma = data['TEXT'][doc_indices_zero_gamma]

    relevant_doc_ids = data['DOCNO'][doc_indices]
    relevant_doc_ids_zero_gamma = data['DOCNO'][doc_indices_zero_gamma]

    query_relevant_documents[query_num] = [relevant_doc_ids,relevant_docs]
    query_relevant_documents_zero_gamma[query_num] = [relevant_doc_ids_zero_gamma,relevant_docs_zero_gamma]

In [0]:
query_precision_recall_f1_score = []
for i in query_relevant_documents.keys():
    l = []
    print("Query Number: ",i)
    l.append(i)
    print("Documents retrieved using QUERY ROCCHIO(TF-IDF): gamma = 0.15 ")
    print(query_relevant_documents[i][0])
    print("\nRelevant documents: ")
    for q in query_rel_doc[str(i)]:
        print(q)
    retrieved = set(query_relevant_documents[i][0])
    act_docs = set(query_rel_doc[str(i)])
    precision = len(retrieved.intersection(act_docs))/5
    recall = len(retrieved.intersection(act_docs))/len(act_docs)
    f1_score = 0
    if precision+recall != 0:
      f1_score = 2*precision*recall/(precision+recall)
    l.extend([precision,recall,f1_score])
    query_precision_recall_f1_score.append(l)
    print("\nNumber of relevant documents retrieved: ",len(retrieved.intersection(act_docs)))
    print("Relevant documents retrieved using QUERY ROCCHIO(TF-IDF): gamma = 0.15")
    for d in retrieved.intersection(act_docs):
        print(d)
    print("\n")
    print("*"*100)

Query Number:  76
Documents retrieved using QUERY ROCCHIO(TF-IDF): gamma = 0.15 
0                 1050423_bengal_index.utf8
1875      1050423_bengal_story_4651659.utf8
2338      1050426_bengal_story_4662636.utf8
20818    1050423_opinion_story_4650147.utf8
13204     1050422_nation_story_4643866.utf8
Name: DOCNO, dtype: object

Relevant documents: 
1070530_nation_story_7849973.utf8
1070602_nation_story_7865940.utf8
1070603_nation_story_7869357.utf8
1070611_nation_story_7906812.utf8

Number of relevant documents retrieved:  0
Relevant documents retrieved using QUERY ROCCHIO(TF-IDF): gamma = 0.15


****************************************************************************************************
Query Number:  77
Documents retrieved using QUERY ROCCHIO(TF-IDF): gamma = 0.15 
1           1050303_bengal_story_4447924.utf8
2210        1050316_bengal_story_4498474.utf8
597         1050226_bengal_story_4427183.utf8
31819    1050325_frontpage_story_4535852.utf8
573         1050321_bengal_stor

In [0]:
query_precision_recall_f1_score_zero_gamma = []
for i in query_relevant_documents_zero_gamma.keys():
    l = []
    print("Query Number: ",i)
    l.append(i)
    print("Documents retrieved using QUERY ROCCHIO(TF-IDF) zero gamma: ")
    print(query_relevant_documents[i][0])
    print("\nRelevant documents: ")
    for q in query_rel_doc[str(i)]:
        print(q)
    retrieved = set(query_relevant_documents[i][0])
    act_docs = set(query_rel_doc[str(i)])
    precision = len(retrieved.intersection(act_docs))/5
    recall = len(retrieved.intersection(act_docs))/len(act_docs)
    f1_score = 0
    if precision+recall != 0:
      f1_score = 2*precision*recall/(precision+recall)
    l.extend([precision,recall,f1_score])
    query_precision_recall_f1_score_zero_gamma.append(l)
    print("\nNumber of relevant documents retrieved: ",len(retrieved.intersection(act_docs)))
    print("Relevant documents retrieved using QUERY ROCCHIO(TF-IDF) zero gamma:")
    for d in retrieved.intersection(act_docs):
        print(d)
    print("\n")
    print("*"*100)

Query Number:  76
Documents retrieved using QUERY ROCCHIO(TF-IDF) zero gamma: 
0                 1050423_bengal_index.utf8
1875      1050423_bengal_story_4651659.utf8
2338      1050426_bengal_story_4662636.utf8
20818    1050423_opinion_story_4650147.utf8
13204     1050422_nation_story_4643866.utf8
Name: DOCNO, dtype: object

Relevant documents: 
1070530_nation_story_7849973.utf8
1070602_nation_story_7865940.utf8
1070603_nation_story_7869357.utf8
1070611_nation_story_7906812.utf8

Number of relevant documents retrieved:  0
Relevant documents retrieved using QUERY ROCCHIO(TF-IDF) zero gamma:


****************************************************************************************************
Query Number:  77
Documents retrieved using QUERY ROCCHIO(TF-IDF) zero gamma: 
1           1050303_bengal_story_4447924.utf8
2210        1050316_bengal_story_4498474.utf8
597         1050226_bengal_story_4427183.utf8
31819    1050325_frontpage_story_4535852.utf8
573         1050321_bengal_story_4518

In [0]:
avg_precision = 0
avg_recall = 0
avg_f1_score = 0

for query_no,precision,recall,f1_score in query_precision_recall_f1_score:
  avg_precision+=precision
  avg_recall+=recall
  avg_f1_score+=f1_score

print('Average Precison: {} Average Recall: {} Average F1-Score: {}'.format(avg_precision/len(query_precision_recall_f1_score),
                                                                            avg_recall/len(query_precision_recall_f1_score),
                                                                            avg_f1_score/len(query_precision_recall_f1_score)))

Average Precison: 0.15600000000000003 Average Recall: 0.06558592204972846 Average F1-Score: 0.08289289067718761


In [0]:
avg_precision = 0
avg_recall = 0
avg_f1_score = 0

for query_no,precision,recall,f1_score in query_precision_recall_f1_score_zero_gamma:
  avg_precision+=precision
  avg_recall+=recall
  avg_f1_score+=f1_score

print('Average Precison: {} Average Recall: {} Average F1-Score: {}'.format(avg_precision/len(query_precision_recall_f1_score),
                                                                            avg_recall/len(query_precision_recall_f1_score),
                                                                            avg_f1_score/len(query_precision_recall_f1_score)))

Average Precison: 0.15600000000000003 Average Recall: 0.06558592204972846 Average F1-Score: 0.08289289067718761


# **WORD2VEC + QUERY EXPANSION + ROCCHIO**

In [0]:
from gensim.test.utils import datapath
from gensim import utils

class MyCorpus(object):
    """An interator that yields sentences (lists of str)."""
    def __iter__(self): 
        preprocessed_text_column = preprocessed_data['TEXT'].values
        for line in preprocessed_text_column:
            # assume there's one document per line, tokens separated by whitespace
            yield utils.simple_preprocess(line)

In [0]:
import gensim.models
#skipgram model
sentences = MyCorpus()
skipGramModel = gensim.models.Word2Vec(sentences=sentences,sg = 1,workers=4)

In [0]:
import numpy as np
query_vectors = []
for k in range(50):

    #generating vector for each query by summing word vectors
    query = expanded_query_dataframe['query_desc'][k]
    query_words = query.split()
    query_vector = np.zeros(100)
    count = 0
    for l in range(len(query_words)):
      try:
        query_vector+=skipGramModel.wv.word_vec(query_words[l])
        count+=1
      except:
        pass
    if count > 1:
      query_vector/=count
    query_vectors.append(query_vector)


In [0]:
doc_vectors = []
for k in range(preprocessed_data.shape[0]):
#generating vector for each document by summing word vectors
    doc = preprocessed_data['TEXT'][k]
    doc_words = doc.split()
    doc_vector = np.zeros(100)
    count = 0
    for l in range(len(doc_words)):
      try:
        doc_vector+=skipGramModel.wv.word_vec(doc_words[l])
        count+=1
      except:
        pass
    if count > 1:
      doc_vector/=count
    doc_vectors.append(doc_vector)

In [0]:
import numpy as np
sum_of_document_vectors = np.zeros(100)
for i in range(len(doc_vectors)):
  sum_of_document_vectors+=doc_vectors[i]

In [0]:
query_vectors_rel_feedback = []
query_vectors_rel_zero_gamma_feedback = []
#Implementing rocchio algorithm 
for i in range(50):
  query_num = query_data['query_no'][i]
  document_ids = query_rel_doc[str(query_num)]
  sum_of_relevant_doc_vectors = np.zeros(100)
  for doc_id in document_ids:
    for j in range(preprocessed_data.shape[0]):
      if doc_id == preprocessed_data['DOCNO'][j]:
        sum_of_relevant_doc_vectors+=doc_vectors[i]
  avg_rel_doc_vectors = sum_of_relevant_doc_vectors/len(document_ids)
  avg_irrelevant_vectors = (sum_of_document_vectors - sum_of_relevant_doc_vectors)/(preprocessed_data.shape[0] - len(document_ids))
  
  # alpha = 1, beta = 0.75 and gamma = 0.15 
  final_query_vector = query_vectors[i]+ 0.75 * avg_rel_doc_vectors - 0.15 * avg_irrelevant_vectors
  query_vectors_rel_feedback.append(final_query_vector)

  final_query_vector_zero_gamma = query_vectors[i]+ 0.75 * avg_rel_doc_vectors
  query_vectors_rel_zero_gamma_feedback.append(final_query_vector_zero_gamma)


In [0]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
#performing similarity check between queries and documents and finding top 5 relevant documents
similarity_values = []
similarity_values_zero_gamma = []
query_relevant_documents = dict()
query_relevant_documents_zero_gamma = dict()

for k in range(50):
    query_vector = query_vectors_rel_feedback[k]
    query_vector_zero_gamma = query_vectors_rel_zero_gamma_feedback[k]
    similarity_values = []
    for i in range(preprocessed_data.shape[0]):  
        try:
          value = cosine_similarity([query_vector],[doc_vectors[i]])[0][0]
          value_zero_gamma = cosine_similarity([query_vector_zero_gamma],[doc_vectors[i]])[0][0]
        except:
          break
        similarity_values.append([value,i])
        similarity_values_zero_gamma.append([value_zero_gamma,i])
   
    query_num = query_data['query_no'][k]

    similarity_values.sort(reverse = True)
    similarity_values_zero_gamma.sort(reverse = True)

    top_5 = similarity_values[0:5]
    top_5_zero_gamma = similarity_values_zero_gamma[0:5]

    doc_indices = [ i[1] for i in top_5 ]
    doc_indices_zero_gamma = [ i[1] for i in top_5_zero_gamma ]

    relevant_docs = data['TEXT'][doc_indices]
    relevant_docs_zero_gamma = data['TEXT'][doc_indices_zero_gamma]

    relevant_doc_ids = data['DOCNO'][doc_indices]
    relevant_doc_ids_zero_gamma =  data['DOCNO'][doc_indices_zero_gamma]

    query_relevant_documents[query_num] = [relevant_doc_ids,relevant_docs]
    query_relevant_documents_zero_gamma[query_num] = [relevant_doc_ids_zero_gamma,relevant_docs_zero_gamma]

In [21]:
query_precision_recall_f1_score = []
for i in query_relevant_documents.keys():
    l = []
    print("Query Number: ",i)
    l.append(i)
    print("Documents retrieved using QUERY EXPANSION ROCCHIO(word2vec): ")
    print(query_relevant_documents[i][0])
    print("\nRelevant documents: ")
    for q in query_rel_doc[str(i)]:
        print(q)
    retrieved = set(query_relevant_documents[i][0])
    act_docs = set(query_rel_doc[str(i)])
    precision = len(retrieved.intersection(act_docs))/5
    recall = len(retrieved.intersection(act_docs))/len(act_docs)
    f1_score = 0
    if precision+recall != 0:
      f1_score = 2*precision*recall/(precision+recall)
    l.extend([precision,recall,f1_score])
    query_precision_recall_f1_score.append(l)
    print("\nNumber of relevant documents retrieved: ",len(retrieved.intersection(act_docs)))
    print("Relevant documents retrieved using QUERY EXPANSION ROCCHIO(word2vec):")
    for d in retrieved.intersection(act_docs):
        print(d)
    print("\n")
    print("*"*100)

Query Number:  76
Documents retrieved using QUERY EXPANSION ROCCHIO(word2vec): 
12596        1050808_nation_story_5085846.utf8
53982        1060422_nation_story_6130818.utf8
2302         1050909_bengal_story_5216329.utf8
26817      1050622_calcutta_story_4897785.utf8
101382    1070314_frontpage_story_7514044.utf8
Name: DOCNO, dtype: object

Relevant documents: 
1070530_nation_story_7849973.utf8
1070602_nation_story_7865940.utf8
1070603_nation_story_7869357.utf8
1070611_nation_story_7906812.utf8

Number of relevant documents retrieved:  0
Relevant documents retrieved using QUERY EXPANSION ROCCHIO(word2vec):


****************************************************************************************************
Query Number:  77
Documents retrieved using QUERY EXPANSION ROCCHIO(word2vec): 
17890    1051015_foreign_story_5357026.utf8
56411    1060811_foreign_story_6595456.utf8
57084    1060722_foreign_story_6510959.utf8
55598    1060727_foreign_story_6531510.utf8
55401    1060802_foreign_st

In [22]:
query_precision_recall_f1_score_zero_gamma = []
for i in query_relevant_documents_zero_gamma.keys():
    l = []
    print("Query Number: ",i)
    l.append(i)
    print("Documents retrieved using QUERY EXPANSION ROCCHIO(word2vec) gamma = 0: ")
    print(query_relevant_documents[i][0])
    print("\nRelevant documents: ")
    for q in query_rel_doc[str(i)]:
        print(q)
    retrieved = set(query_relevant_documents[i][0])
    act_docs = set(query_rel_doc[str(i)])
    precision = len(retrieved.intersection(act_docs))/5
    recall = len(retrieved.intersection(act_docs))/len(act_docs)
    f1_score = 0
    if precision+recall != 0:
      f1_score = 2*precision*recall/(precision+recall)
    l.extend([precision,recall,f1_score])
    query_precision_recall_f1_score_zero_gamma.append(l)
    print("\nNumber of relevant documents retrieved: ",len(retrieved.intersection(act_docs)))
    print("Relevant documents retrieved using QUERY EXPANSION ROCCHIO(word2vec) gamma = 0:")
    for d in retrieved.intersection(act_docs):
        print(d)
    print("\n")
    print("*"*100)

Query Number:  76
Documents retrieved using QUERY EXPANSION ROCCHIO(word2vec) gamma = 0: 
12596        1050808_nation_story_5085846.utf8
53982        1060422_nation_story_6130818.utf8
2302         1050909_bengal_story_5216329.utf8
26817      1050622_calcutta_story_4897785.utf8
101382    1070314_frontpage_story_7514044.utf8
Name: DOCNO, dtype: object

Relevant documents: 
1070530_nation_story_7849973.utf8
1070602_nation_story_7865940.utf8
1070603_nation_story_7869357.utf8
1070611_nation_story_7906812.utf8

Number of relevant documents retrieved:  0
Relevant documents retrieved using QUERY EXPANSION ROCCHIO(word2vec) gamma = 0:


****************************************************************************************************
Query Number:  77
Documents retrieved using QUERY EXPANSION ROCCHIO(word2vec) gamma = 0: 
17890    1051015_foreign_story_5357026.utf8
56411    1060811_foreign_story_6595456.utf8
57084    1060722_foreign_story_6510959.utf8
55598    1060727_foreign_story_6531510.ut

In [23]:
avg_precision = 0
avg_recall = 0
avg_f1_score = 0

for query_no,precision,recall,f1_score in query_precision_recall_f1_score:
  avg_precision+=precision
  avg_recall+=recall
  avg_f1_score+=f1_score

print('Average Precison: {} Average Recall: {} Average F1-Score: {}'.format(avg_precision/len(query_precision_recall_f1_score),
                                                                            avg_recall/len(query_precision_recall_f1_score),
                                                                            avg_f1_score/len(query_precision_recall_f1_score)))

Average Precison: 0.15200000000000002 Average Recall: 0.05767384011053715 Average F1-Score: 0.07554054500657567


In [24]:
avg_precision = 0
avg_recall = 0
avg_f1_score = 0

for query_no,precision,recall,f1_score in query_precision_recall_f1_score_zero_gamma:
  avg_precision+=precision
  avg_recall+=recall
  avg_f1_score+=f1_score

print('Average Precison: {} Average Recall: {} Average F1-Score: {}'.format(avg_precision/len(query_precision_recall_f1_score),
                                                                            avg_recall/len(query_precision_recall_f1_score),
                                                                            avg_f1_score/len(query_precision_recall_f1_score)))

Average Precison: 0.15200000000000002 Average Recall: 0.05767384011053715 Average F1-Score: 0.07554054500657567


# **SUMMARY**

In [26]:
from prettytable import PrettyTable
    
x = PrettyTable()

x.field_names = ["METHOD", "PRECISION", "RECALL", "F1 SCORE"]

x.add_row(["QUERY EXPANSION + TFIDF ", 0.284, 0.1298, 0.1598])
x.add_row(["QUERY EXPANSION + WORD2VEC", 0.152, 0.061, 0.078])
x.add_row(["QUERY EXPANSION + TFIDF + ROCCHIO (GAMMA = 0.15)", 0.15, 0.0655, 0.0828])
x.add_row(["QUERY EXPANSION + TFIDF + ROCCHIO (GAMMA = 0)", 0.15, 0.0655, 0.0828])
x.add_row(["QUERY EXPANSION + WORD2VEC + ROCCHIO (GAMMA = 0.15)", 0.15, 0.0576, 0.0755])
x.add_row(["QUERY EXPANSION + WORD2VEC + ROCCHIO (GAMMA = 0)", 0.15, 0.0576, 0.0755])

x.add_row(["TFIDF + ROCCHIO (GAMMA = 0.15)", 0.284, 0.155, 0.182])
x.add_row(["TFIDF + ROCCHIO (GAMMA = 0)", 0.284, 0.155, 0.182])
x.add_row(["WORD2VEC + ROCCHIO (GAMMA = 0.15)", 0.26, 0.122, 0.152])
x.add_row(["WORD2VEC + ROCCHIO (GAMMA = 0)", 0.26, 0.122, 0.152])


print(x)

+-----------------------------------------------------+-----------+--------+----------+
|                        METHOD                       | PRECISION | RECALL | F1 SCORE |
+-----------------------------------------------------+-----------+--------+----------+
|               QUERY EXPANSION + TFIDF               |   0.284   | 0.1298 |  0.1598  |
|              QUERY EXPANSION + WORD2VEC             |   0.152   | 0.061  |  0.078   |
|   QUERY EXPANSION + TFIDF + ROCCHIO (GAMMA = 0.15)  |    0.15   | 0.0655 |  0.0828  |
|    QUERY EXPANSION + TFIDF + ROCCHIO (GAMMA = 0)    |    0.15   | 0.0655 |  0.0828  |
| QUERY EXPANSION + WORD2VEC + ROCCHIO (GAMMA = 0.15) |    0.15   | 0.0576 |  0.0755  |
|   QUERY EXPANSION + WORD2VEC + ROCCHIO (GAMMA = 0)  |    0.15   | 0.0576 |  0.0755  |
|            TFIDF + ROCCHIO (GAMMA = 0.15)           |   0.284   | 0.155  |  0.182   |
|             TFIDF + ROCCHIO (GAMMA = 0)             |   0.284   | 0.155  |  0.182   |
|          WORD2VEC + ROCCHIO (G

**It can be observed that rocchio algorithm is helpful in improving recall and query expansion does not seem to improve the results.**